# WI-Projekt
Nr. 18

... <br>
... <br>
BA.WIN.22HS.VZa

... <br>
... <br>
BA.WIN.22HS.VZb

Pascal Kern <br>
kernpas1@students.zhaw.ch <br>
BA.WIN.22HS.VZa

## Setup

#### Get required Packages

In [1]:
import json
import os
import boto3 # für Bedrock, funktioniert nicht

from huggingface_hub import login # Für LLama
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

credentials = "credentials.json"

with open(credentials, 'r') as f:
    credentials = json.load(f)

# API-Schlüssel setzen
os.environ["OPENAI_API_KEY"] = credentials['openai']['api_key'] # OpenAI
os.environ["GEMINI_API_KEY"] = credentials['gemini']['api_key'] # Gemini

# aws Bedrock
os.environ["AWS_ACCESS_KEY_ID"] = credentials["awsBedrock"]["aws_access_key_id"]
os.environ["AWS_SECRET_ACCESS_KEY"] = credentials["awsBedrock"]["aws_secret_access_key"] 
region_name=credentials["awsBedrock"]['region'] 
# Initialisiere den boto3 Client für Bedrock
bedrock_client = boto3.client(
    'bedrock-runtime',  
    region_name=credentials["awsBedrock"]['region']  # Region aus deinen credentials
)

# Huggingface llama
huggingface_token = credentials['huggingface']['api_key']
login(huggingface_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\pasca\.cache\huggingface\token
Login successful


#### Connect to DB

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'AntÃ´nio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

### AWS Bedrock

In [3]:
'''
from langchain.llms.base import LLM
from pydantic import BaseModel
from typing import Optional, List
import json

class BedrockLLM(LLM, BaseModel):
    client: any  # Der Bedrock-Client (boto3)
    model_id: str  # Modell-ID des Bedrock-Modells
    region: str  # AWS-Region, in der das Bedrock-Modell gehostet wird

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        # Bereite den Anfrage-Body vor
        body = {
            "inputText": prompt
        }

        # Mache den API-Aufruf zum Bedrock-Modell
        response = self.client.invoke_model(
            modelId=self.model_id,
            contentType="application/json",
            accept="application/json",  # Ggf. anpassen
            body=json.dumps(body)  # Den Body in JSON-Format konvertieren
        )

        # Debug-Ausgabe der Antwort hinzufügen
        print("Raw API response:", response)

        # Entpacke die Antwort und prüfe die Struktur
        result = json.loads(response['body'].read())
        print("Decoded response:", result)

        # Sicherstellen, dass das richtige Feld für die Ausgabe verwendet wird
        if 'outputText' in result:
            return result['outputText']
        else:
            return f"Fehler: Kein 'outputText' gefunden, API-Antwort: {result}"

    @property
    def _llm_type(self) -> str:
        return "aws-bedrock"
'''


'\nfrom langchain.llms.base import LLM\nfrom pydantic import BaseModel\nfrom typing import Optional, List\nimport json\n\nclass BedrockLLM(LLM, BaseModel):\n    client: any  # Der Bedrock-Client (boto3)\n    model_id: str  # Modell-ID des Bedrock-Modells\n    region: str  # AWS-Region, in der das Bedrock-Modell gehostet wird\n\n    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:\n        # Bereite den Anfrage-Body vor\n        body = {\n            "inputText": prompt\n        }\n\n        # Mache den API-Aufruf zum Bedrock-Modell\n        response = self.client.invoke_model(\n            modelId=self.model_id,\n            contentType="application/json",\n            accept="application/json",  # Ggf. anpassen\n            body=json.dumps(body)  # Den Body in JSON-Format konvertieren\n        )\n\n        # Debug-Ausgabe der Antwort hinzufügen\n        print("Raw API response:", response)\n\n        # Entpacke die Antwort und prüfe die Struktur\n        result =

In [4]:
'''
# Initialize the Bedrock LLM with the client and model information
llm = BedrockLLM(client=bedrock_client, model_id="amazon.titan-embed-text-v1", region=region_name)

# Now you can use this LLM in your LangChain pipeline
Bedrock_agent = create_sql_agent(llm, db=db, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True)
'''


'\n# Initialize the Bedrock LLM with the client and model information\nllm = BedrockLLM(client=bedrock_client, model_id="amazon.titan-embed-text-v1", region=region_name)\n\n# Now you can use this LLM in your LangChain pipeline\nBedrock_agent = create_sql_agent(llm, db=db, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True)\n'

In [5]:
'''
Bedrock_agent.invoke(
    "Welcher Artist hat am meisten Albums?"
)
'''

'\nBedrock_agent.invoke(\n    "Welcher Artist hat am meisten Albums?"\n)\n'

### Agent OpenAI

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0) ## Which Large Language model are we using?
toolkit = SQLDatabaseToolkit(llm=llm, db=db)

OpenAI_agent = create_sql_agent(llm, toolkit=toolkit, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True) 


### Agent Google Gemini

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0.7, api_key=os.getenv("GEMINI_API_KEY")) ## mit temperatur 0.7 Gemini gives the same awnser like Openai when not using an agent type
Gemini_agent = create_sql_agent(llm, db=db, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True) 

### Agent Meta LLaMA

In [8]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline

# Load LLaMA model from Hugging Face
model_id = "meta-llama/Llama-3.2-1B"  
pipe = pipeline("text-generation", model=model_id, max_new_tokens=100, max_length=100, num_return_sequences=1, temperature=0.7)

# Create a LangChain-compatible LLM using the HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)
LLaMA_agent = create_sql_agent(llm, db=db, verbose=True) 

### Execute

In [9]:
OpenAI_agent.invoke(
    "Welcher Artist hat am meisten Albums?"
)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackTo determine which artist has the most albums, I should examine the "Album" and "Artist" tables. I will first check the schema of these tables to understand their structure and relationships.

Action: sql_db_schema
Action Input: "Album, Artist"
CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/To find out which artist has the

{'input': 'Welcher Artist hat am meisten Albums?',
 'output': 'The artist with the most albums is Iron Maiden, with 21 albums.'}

In [10]:
Gemini_agent.invoke(
    "Welcher Artist hat am meisten Albums?"
)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackI should query the schema of the Artist and Album tables.
Action: sql_db_schema
Action Input: Album, Artist
CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/I should find the total number of albums for each artist, order by count descending, and get the artist with the highest count.
Action: sql_db_query
Action Input: SELECT A

{'input': 'Welcher Artist hat am meisten Albums?', 'output': 'Iron Maiden'}

In [11]:
'''
LLaMA_agent.invoke(
    "Welcher Artist hat am meisten Albums?"
)
'''

'\nLLaMA_agent.invoke(\n    "Welcher Artist hat am meisten Albums?"\n)\n'